In [1]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-rqluir4r
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-rqluir4r
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369549 sha256=400b668b09ee4cffb34002c25e7da9fef6a4d3061d1c399af51c0092ace857f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-ob968las/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [2]:
!wget -O image.jpg "https://content.presspage.com/uploads/2170/1920_face-new-675956.jpg?10000"
!wget -O image-2.jpg "https://as2.ftcdn.net/jpg/01/41/14/71/1000_F_141147103_DMr5f66cIrRUpbyFOMmz4MsVC0gMeD5i.jpg"
!wget -O image-3.jpg "https://as1.ftcdn.net/v2/jpg/03/42/85/16/1000_F_342851651_BV9SYWnoTPSu3kq6e82zG7H8eEt20wd1.jpg"


--2025-12-29 15:08:30--  https://content.presspage.com/uploads/2170/1920_face-new-675956.jpg?10000
Resolving content.presspage.com (content.presspage.com)... 143.204.160.111, 143.204.160.81, 143.204.160.19, ...
Connecting to content.presspage.com (content.presspage.com)|143.204.160.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 164847 (161K) [image/jpeg]
Saving to: ‘image.jpg’

image.jpg           100%[===================>] 160.98K  --.-KB/s    in 0.02s   

2025-12-29 15:08:30 (10.1 MB/s) - ‘image.jpg’ saved [164847/164847]

--2025-12-29 15:08:30--  https://as2.ftcdn.net/jpg/01/41/14/71/1000_F_141147103_DMr5f66cIrRUpbyFOMmz4MsVC0gMeD5i.jpg
Resolving as2.ftcdn.net (as2.ftcdn.net)... 151.101.1.91, 151.101.65.91, 151.101.129.91, ...
Connecting to as2.ftcdn.net (as2.ftcdn.net)|151.101.1.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223668 (218K) [image/jpeg]
Saving to: ‘image-2.jpg’

image-2.jpg         100%[===================>

In [3]:
# @title TEST CLIP
file_name = "image-3.jpg" # @param {"type":"string"}
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open(file_name)).unsqueeze(0).to(device)

emotions = [
    "happy",
    "sad",
    "angry",
    "fear",
    "surprised",
    "disgust",
    "neutral"
]

prompts = [f"a photo of a person feeling {e}" for e in emotions]

text = clip.tokenize(prompts).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu()

pred_idx = probs.argmax(axis=-1)
print(f"Predicted emotion: {prompts[pred_idx]} [with a probability of {probs[:, pred_idx].item()}]")

100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 189MiB/s]


Predicted emotion: a photo of a person feeling neutral [with a probability of 0.23908425867557526]
